# Bottleneck Analysis

In [ ]:
# import time
# import cv2
# import numpy as np
# import pandas as pd
# import psutil
# import cProfile
# import pstats
# from io import StringIO
# from ML_KEM import ML_KEM
# from AES_CTR.AES_Lib import AES_Software

# # --- CẤU HÌNH ANALYSIS ---
# TEST_DURATION_SEC = 10    # Chạy test trong 10 giây
# RESOLUTION = (640, 480)   # Test ở VGA
# ENABLE_PROFILING = True   # Bật "máy soi" code

# print("=== SYSTEM BOTTLENECK ANALYSIS ===")

# # 1. SETUP CHUNG
# # Sinh khóa 1 lần
# pk, sk = ML_KEM.KeyGen()
# ss, _ = ML_KEM.Encaps(pk)
# aes = AES_Software(ss)

# # Tạo dữ liệu giả lập (Synthetic Data)
# dummy_frame = np.random.randint(0, 256, (RESOLUTION[1], RESOLUTION[0], 3), dtype=np.uint8)

# # Hàm tiện ích để đo CPU usage
# def get_cpu_usage():
#     return psutil.cpu_percent(interval=None)

# # ==========================================
# # TEST CASE 1: SYNTHETIC INPUT (COMPUTE ONLY)
# # Loại bỏ hoàn toàn độ trễ của Camera, chỉ đo sức mạnh tính toán thuần túy
# # ==========================================
# print("\n>> RUNNING TEST 1: PURE COMPUTE (SYNTHETIC INPUT)...")

# frame_count = 0
# start_time = time.time()
# cpu_stats = []

# # Bật Profiler nếu cần
# pr = cProfile.Profile()
# if ENABLE_PROFILING:
#     pr.enable()

# while (time.time() - start_time) < TEST_DURATION_SEC:
#     # INPUT: Lấy trực tiếp từ RAM (Cực nhanh)
#     frame = dummy_frame 
    
#     # PROCESS: Encrypt + Decrypt
#     enc = aes.encrypt_image(frame)
#     dec = aes.decrypt_to_image(enc, frame.shape, frame.dtype)
    
#     frame_count += 1
#     if frame_count % 10 == 0:
#         cpu_stats.append(get_cpu_usage())

# if ENABLE_PROFILING:
#     pr.disable()

# fps_synthetic = frame_count / TEST_DURATION_SEC
# avg_cpu_synthetic = np.mean(cpu_stats) if cpu_stats else 0

# print(f"-> Synthetic FPS: {fps_synthetic:.2f}")
# print(f"-> Avg CPU Usage: {avg_cpu_synthetic:.1f}%")

# # ==========================================
# # TEST CASE 2: REAL CAMERA (I/O + COMPUTE)
# # Đo hệ thống trong điều kiện thực tế
# # ==========================================
# print("\n>> RUNNING TEST 2: REAL-WORLD (CAMERA I/O)...")
# cap = cv2.VideoCapture(0)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, RESOLUTION[0])
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, RESOLUTION[1])

# frame_count = 0
# start_time = time.time()
# cpu_stats = []

# while (time.time() - start_time) < TEST_DURATION_SEC:
#     # INPUT: Đọc từ Camera (Chậm hơn RAM)
#     ret, frame = cap.read()
#     if not ret: break
    
#     # Resize đảm bảo đúng size test
#     if frame.shape[:2] != (RESOLUTION[1], RESOLUTION[0]):
#         frame = cv2.resize(frame, RESOLUTION)

#     # PROCESS
#     enc = aes.encrypt_image(frame)
#     dec = aes.decrypt_to_image(enc, frame.shape, frame.dtype)
    
#     frame_count += 1
#     if frame_count % 10 == 0:
#         cpu_stats.append(get_cpu_usage())

# fps_real = frame_count / TEST_DURATION_SEC
# avg_cpu_real = np.mean(cpu_stats) if cpu_stats else 0
# cap.release()

# print(f"-> Real Camera FPS: {fps_real:.2f}")
# print(f"-> Avg CPU Usage: {avg_cpu_real:.1f}%")

# # ==========================================
# # 3. ANALYSIS REPORT & PROFILING RESULT
# # ==========================================
# print("\n" + "="*40)
# print("      BOTTLENECK DIAGNOSIS REPORT      ")
# print("="*40)

# # A. So sánh I/O vs Compute
# io_overhead = fps_synthetic - fps_real
# print(f"1. Max Potential FPS (Compute Limit): {fps_synthetic:.2f}")
# print(f"2. Actual FPS (With Camera):          {fps_real:.2f}")
# print(f"3. FPS Loss due to I/O (Camera):      {io_overhead:.2f}")

# if avg_cpu_synthetic > 85:
#     print("=> CONCLUSION: System is COMPUTE BOUND (CPU bị quá tải).")
#     print("   Đây là ứng viên hoàn hảo để dùng FPGA Acceleration.")
# elif io_overhead > (fps_synthetic * 0.5):
#     print("=> CONCLUSION: System is I/O BOUND (Camera quá chậm).")
#     print("   FPGA vẫn giúp giảm CPU load, nhưng FPS có thể không tăng nhiều nếu không đổi Camera.")

# # B. Chi tiết Profiling (Hàm nào chậm nhất?)
# if ENABLE_PROFILING:
#     print("\n[PROFILING RESULT - TOP 10 SLOWEST FUNCTIONS]")
#     s = StringIO()
#     # Sắp xếp theo 'cumtime' (thời gian tích lũy) để thấy hàm nào chiếm tổng thời gian nhiều nhất
#     sortby = 'cumtime' 
#     ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
#     ps.print_stats(10) # In ra top 10 dòng
#     print(s.getvalue())

In [ ]:
import time
import cv2
import numpy as np
import psutil
import cProfile
import pstats
from io import StringIO
from ML_KEM import ML_KEM
from AES_CTR.AES_Lib import AES_Software

print("=== PHASE 1: DEEP-DIVE BOTTLENECK ANALYSIS ===")

def print_profile_stats(pr, title, top_n=15):
    """Hàm hỗ trợ in kết quả Profile đẹp mắt"""
    s = StringIO()
    sortby = 'cumtime' # Sắp xếp theo thời gian tích lũy
    ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
    ps.print_stats(top_n)
    print(f"\n--- {title} TOP {top_n} SLOWEST FUNCTIONS ---")
    print(s.getvalue())

# ========================================================
# SECTION 1: KYBER PROFILING (Tìm điểm nghẽn thuật toán)
# Mục tiêu: Chứng minh NTT và Hash là bottleneck cần FPGA
# ========================================================
print(f"\n>> [1/2] PROFILING KYBER ALGORITHMS...")

# 1.1 Profile KeyGen
print("   -> Profiling KeyGen...")
pr = cProfile.Profile()
pr.enable()
for _ in range(100): # Chạy 100 lần để lấy mẫu đủ lớn
    ML_KEM.KeyGen()
pr.disable()
print_profile_stats(pr, "KYBER KEYGEN")

# 1.2 Profile Encaps
pk, sk = ML_KEM.KeyGen()
print("   -> Profiling Encaps...")
pr = cProfile.Profile()
pr.enable()
for _ in range(100):
    ML_KEM.Encaps(pk)
pr.disable()
print_profile_stats(pr, "KYBER ENCAPS")

# 1.3 Profile Decaps
ss, c = ML_KEM.Encaps(pk)
print("   -> Profiling Decaps...")
pr = cProfile.Profile()
pr.enable()
for _ in range(100):
    ML_KEM.Decaps(sk, c)
pr.disable()
print_profile_stats(pr, "KYBER DECAPS")

print("\n=> HINT: Look for 'NTT.py', 'SHA3.py' or 'CryptoFunc.py' in the lists above.")

# ========================================================
# SECTION 2: SYSTEM I/O vs COMPUTE BOTTLENECK
# Mục tiêu: Xác định CPU hay Camera đang làm chậm hệ thống
# ========================================================
print(f"\n>> [2/2] ANALYZING SYSTEM BOTTLENECK (CPU vs I/O)...")

# Cấu hình test Stress
TEST_DURATION = 10 # giây
RES = (640, 480)
aes = AES_Software(ss)

# A. Test Compute Bound (Input từ RAM - Tốc độ tối đa của CPU)
print("   -> Running Synthetic Input Test (CPU Limit)...")
dummy = np.random.randint(0, 256, (RES[1], RES[0], 3), dtype=np.uint8)
frames_syn = 0
start_t = time.time()
cpu_log = []

while time.time() - start_t < TEST_DURATION:
    aes.decrypt_to_image(aes.encrypt_image(dummy), dummy.shape, dummy.dtype)
    frames_syn += 1
    if frames_syn % 5 == 0: cpu_log.append(psutil.cpu_percent())

fps_syn = frames_syn / TEST_DURATION
avg_cpu_syn = np.mean(cpu_log)

# B. Test I/O Bound (Input từ Camera - Tốc độ thực tế)
print("   -> Running Real Camera Test (I/O Constraint)...")
cap = cv2.VideoCapture(0)
has_cam = cap.isOpened()
if has_cam:
    frames_real = 0
    start_t = time.time()
    while time.time() - start_t < TEST_DURATION:
        ret, frame = cap.read()
        if not ret: break
        frame = cv2.resize(frame, RES)
        aes.decrypt_to_image(aes.encrypt_image(frame), frame.shape, frame.dtype)
        frames_real += 1
    fps_real = frames_real / TEST_DURATION
    cap.release()
else:
    fps_real = 0
    print("      (Skipped: No Camera)")

# Report System Analysis
print("\n[TABLE 3] BOTTLENECK DIAGNOSIS")
print(f"1. Max CPU Capability (FPS):   {fps_syn:.2f} FPS")
print(f"2. Real-world Performance:     {fps_real:.2f} FPS")
print(f"3. FPS Loss due to Camera I/O: {fps_syn - fps_real:.2f} FPS")
print(f"4. Average CPU Usage (Stress): {avg_cpu_syn:.1f}%")

if avg_cpu_syn > 85:
    print("=> CONCLUSION: COMPUTE BOUND. CPU is the bottleneck -> Needs FPGA.")
else:
    print("=> CONCLUSION: I/O BOUND. Camera is slow, but FPGA will still lower Latency.")